In [34]:
import numpy as np
import scipy
import scipy.stats as sts

In [36]:
sts.norm.ppf(1-0.003/2)

2.9677379253417944

___
**В пятилетнем** рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
asp = 104/11037
noasp = 189/11034

In [4]:
noasp - asp

0.0077060239760047815

___
**Постройте** теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его **верхняя граница**? Округлите ответ до четырёх знаков после десятичной точки.

In [37]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [105]:
asp_sample = np.concatenate((np.ones(104), np.zeros(11037-104)))
noasp_sample = np.concatenate((np.ones(189), np.zeros(11034-189)))

print(proportions_confint_diff_ind(noasp_sample, asp_sample, 0.05))

(0.004687750675049439, 0.010724297276960124)


___
**Продолжим** анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить $\hat p$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

$$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189} \approx 0.0174$$

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [17]:
def odds(binary_sample):
    odds = sum(binary_sample) / (len(binary_sample) - sum(binary_sample))
    return odds

In [50]:
a = odds(noasp_sample)
b = odds(asp_sample)
print(a, '\n', b)

0.017427385892116183 
 0.009512485136741973


In [41]:
print(f'{a / b:.4f}')

1.8321


___
**Величина**, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

 - составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
 - установите random seed=0;
 - сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [10]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [28]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [51]:
asp_sample

array([1., 1., 1., ..., 0., 0., 0.])

In [52]:
noasp_sample

array([1., 1., 1., ..., 0., 0., 0.])

In [104]:
np.random.seed(0)

asp_odds_scores = np.apply_along_axis(
    odds, 1, get_bootstrap_samples(asp_sample, 1000))

noasp_odds_scores = np.apply_along_axis(
    odds, 1, get_bootstrap_samples(noasp_sample, 1000))

delta_median_scores = noasp_odds_scores / asp_odds_scores

print(stat_intervals(delta_median_scores, 0.05))